In [117]:
# import bw2regional as br
import bw2data as bd
import bw2calc as bc

In [29]:
bd.projects.set_current("shared scale")

In [104]:
for db_name in list(bd.databases):
    del bd.databases[db_name]
    
for method in list(bd.methods):
    del bd.methods[method]

In [105]:
db = bd.Database("shared-scale")
db.register(geocollections=["watersheds"])

In [106]:
# br.geocollections["watersheds"] = {}

In [107]:
first = db.new_activity(name="first", code="1", location=("watersheds", "a"))
second = db.new_activity(name="second", code="2", location=("watersheds", "b"))
water = db.new_activity(name="water", code="h2o", type="emission")
gw = db.new_activity(name="groundwater", code="gw", type="emission")
first.save(), second.save(), water.save(), gw.save()
first.new_exchange(input=second, type="technosphere", amount=10).save()
first.new_exchange(input=water, type="biosphere", amount=5).save()
second.new_exchange(input=water, type="biosphere", amount=2).save()
second.new_exchange(input=gw, type="biosphere", amount=20).save()

In [108]:
water_consumption = bd.Method(("water consumption", "watershed scale"))
water_consumption.register(unit="DALYs", geocollections=["watersheds"])
water_consumption.write([
    # Stressor, CF amount, location with geocollection
    (water.key, 1e6),  # Implicit global value "GLO"
    (water.key, 2, ("watersheds", "a")),    
    (water.key, 4, ("watersheds", "b")),
])

In [118]:
sg_lca = bc.LCA({first: 1}, ("water consumption", "watershed scale"))
sg_lca.lci()
sg_lca.lcia()
sg_lca.score

25000150.0

In [109]:
lca = br.OneSpatialScaleLCA({first: 1}, ("water consumption", "watershed scale"))

In [110]:
lca.lci()
lca.lcia()

In [111]:
lca.score

90.0

In [112]:
lca.reg_cf_matrix

<2x2 sparse matrix of type '<class 'numpy.float64'>'
	with 2 stored elements in Compressed Sparse Row format>

In [113]:
lca.inv_mapping_matrix

<2x2 sparse matrix of type '<class 'numpy.float64'>'
	with 2 stored elements in Compressed Sparse Row format>

In [114]:
res = lca.results_inv_spatial_scale()
res

<2x2 sparse matrix of type '<class 'numpy.float64'>'
	with 2 stored elements in Compressed Sparse Column format>

In [116]:
lca.biosphere_matrix.toarray()

array([[ 5.,  2.],
       [ 0., 20.]])

In [115]:
res.sum(axis=0)

for location, index in bd.geomapping.items():
    print(location)
    if index in lca.dicts.inv_spatial:
        print(res[:, lca.dicts.inv_spatial[index]].toarray())

GLO
('watersheds', 'a')
[[10.]
 [ 0.]]
('watersheds', 'b')
[[80.]
 [ 0.]]


In [87]:
res.sum(axis=1)

matrix([[90.]])